In [3]:
# Dataset 압축 해제
#!mkdir -p /content/OX_Dataset
!unzip -qq /content/Dataset.zip -d /content/OX_Dataset

In [ ]:
# 라이브러리 임포트
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.utils.data as data
import torchvision.models as models
import numpy as np
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms
import os
from torch.utils.data import Dataset # Import the Dataset class
from PIL import Image
import random

In [ ]:
from sklearn.model_selection import train_test_split

MLP

In [ ]:
# 라이브러리 임포트
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

import torch.nn.functional as F

# 데이터 전처리
mean = 0.5
sigma = 0.5
trans = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Grayscale(),
    transforms.Normalize(mean=[mean], std=[sigma])
])
trainset = datasets.ImageFolder(root='./OX_Dataset/Dataset', transform=trans)

# 데이터 분할
train_data, test_data = train_test_split(trainset, test_size=0.2, shuffle=True)

trainloader = DataLoader(train_data, batch_size=32, shuffle=True)
testloader = DataLoader(test_data, batch_size=32, shuffle=False)

# 모델 정의
class Classification(nn.Module):
    def __init__(self):
        super(Classification, self).__init__()
        self.fc1 = nn.Linear(300*300, 512)  # 300x300 크기의 이미지가 입력
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 2)  # 2개의 클래스 (OX)

    def forward(self, x):
        x = x.view(-1, 300*300)  # 입력을 1D 텐서로 변환
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 모델 초기화
model = Classification()

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 훈련 루프
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in trainloader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}')

# 모델 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f}%')


Epoch [1/10], Loss: 21.5253
Epoch [2/10], Loss: 3.2013
Epoch [3/10], Loss: 3.2655
Epoch [4/10], Loss: 5.1724
Epoch [5/10], Loss: 4.4348
Epoch [6/10], Loss: 2.2466
Epoch [7/10], Loss: 1.0923
Epoch [8/10], Loss: 0.9772
Epoch [9/10], Loss: 0.5421
Epoch [10/10], Loss: 0.5535
Accuracy: 85.71%


MLP(K-fold 적용)

In [ ]:
# 라이브러리 임포트
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold
import numpy as np


import torch.nn.functional as F

# 데이터 전처리
mean = 0.5
sigma = 0.5
trans = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Grayscale(),
    transforms.Normalize(mean=[mean], std=[sigma])
])
dataset = datasets.ImageFolder(root='./OX_Dataset/Dataset', transform=trans)

# K-Fold 설정
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True)

# 모델 정의
class Classification(nn.Module):
    def __init__(self):
        super(Classification, self).__init__()
        self.fc1 = nn.Linear(300*300, 512)  # 300x300 크기의 이미지가 입력
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 2)  # 2개의 클래스 (OX)

    def forward(self, x):
        x = x.view(-1, 300*300)  # 입력을 1D 텐서로 변환
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 손실 함수 정의
criterion = nn.CrossEntropyLoss()

# K-Fold 교차 검증 수행
results = []

for fold, (train_idx, test_idx) in enumerate(kfold.split(dataset)):
    print(f'FOLD {fold}')
    print('--------------------------------')

    # 샘플 인덱스 가져오기
    train_subsampler = Subset(dataset, train_idx)
    test_subsampler = Subset(dataset, test_idx)

    trainloader = DataLoader(train_subsampler, batch_size=32, shuffle=True)
    testloader = DataLoader(test_subsampler, batch_size=32, shuffle=False)

    # 모델 초기화
    model = Classification()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # 훈련 루프
    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in trainloader:
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}')

    # 모델 평가
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy for fold {fold}: {accuracy:.2f}%')
    results.append(accuracy)

# 전체 결과 출력
print(f'K-Fold Cross Validation Results for {k_folds} Folds')
print('--------------------------------')
for i in range(k_folds):
    print(f'Fold {i}: {results[i]:.2f}%')
print(f'Average: {np.mean(results):.2f}%')


FOLD 0
--------------------------------
Epoch [1/10], Loss: 25.7076
Epoch [2/10], Loss: 14.2995
Epoch [3/10], Loss: 6.3132
Epoch [4/10], Loss: 1.0830
Epoch [5/10], Loss: 1.5152
Epoch [6/10], Loss: 1.0961
Epoch [7/10], Loss: 0.6483
Epoch [8/10], Loss: 0.4609
Epoch [9/10], Loss: 0.4467
Epoch [10/10], Loss: 0.4487
Accuracy for fold 0: 57.14%
FOLD 1
--------------------------------
Epoch [1/10], Loss: 18.1662
Epoch [2/10], Loss: 5.5060
Epoch [3/10], Loss: 3.4412
Epoch [4/10], Loss: 1.7142
Epoch [5/10], Loss: 2.0353
Epoch [6/10], Loss: 1.0814
Epoch [7/10], Loss: 0.7246
Epoch [8/10], Loss: 0.5662
Epoch [9/10], Loss: 0.3990
Epoch [10/10], Loss: 0.4677
Accuracy for fold 1: 55.36%
FOLD 2
--------------------------------
Epoch [1/10], Loss: 20.0028
Epoch [2/10], Loss: 5.9150
Epoch [3/10], Loss: 5.2474
Epoch [4/10], Loss: 6.3897
Epoch [5/10], Loss: 2.3308
Epoch [6/10], Loss: 1.4330
Epoch [7/10], Loss: 1.0610
Epoch [8/10], Loss: 0.6554
Epoch [9/10], Loss: 0.5137
Epoch [10/10], Loss: 0.5110
Accurac

CNN(K-fold 적용)

In [5]:
# 라이브러리 임포트
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold
import numpy as np

import torch.nn.functional as F

# 데이터 전처리
mean = 0.5
sigma = 0.5
trans = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Grayscale(),
    transforms.Normalize(mean=[mean], std=[sigma])
])
dataset = datasets.ImageFolder(root='./OX_Dataset/Dataset', transform=trans)

# K-Fold 설정
k_folds = 4
kfold = KFold(n_splits=k_folds, shuffle=True)

# 모델 정의
class Classification(nn.Module):
    def __init__(self):
        super(Classification, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 75 * 75, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 75 * 75)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 손실 함수 정의
criterion = nn.CrossEntropyLoss()

# K-Fold 교차 검증 수행
results = []

for fold, (train_idx, test_idx) in enumerate(kfold.split(dataset)):
    print(f'FOLD {fold}')
    print('--------------------------------')

    # 샘플 인덱스 가져오기
    train_subsampler = Subset(dataset, train_idx)
    test_subsampler = Subset(dataset, test_idx)

    trainloader = DataLoader(train_subsampler, batch_size=32, shuffle=True)
    testloader = DataLoader(test_subsampler, batch_size=32, shuffle=False)

    # 모델 초기화
    model = Classification()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # 훈련 루프
    num_epochs = 10
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in trainloader:
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}')

    # 모델 평가
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy for fold {fold}: {accuracy:.2f}%')
    results.append(accuracy)

# 전체 결과 출력
print(f'K-Fold Cross Validation Results for {k_folds} Folds')
print('--------------------------------')
for i in range(k_folds):
    print(f'Fold {i}: {results[i]:.2f}%')
print(f'Average: {np.mean(results):.2f}%')

FOLD 0
--------------------------------
Epoch [1/10], Loss: 2.7655
Epoch [2/10], Loss: 0.3301
Epoch [3/10], Loss: 0.1017
Epoch [4/10], Loss: 0.0210
Epoch [5/10], Loss: 0.0127
Epoch [6/10], Loss: 0.0036
Epoch [7/10], Loss: 0.0006
Epoch [8/10], Loss: 0.0005
Epoch [9/10], Loss: 0.0001
Epoch [10/10], Loss: 0.0000
Accuracy for fold 0: 78.57%
FOLD 1
--------------------------------
Epoch [1/10], Loss: 3.7479
Epoch [2/10], Loss: 0.3744
Epoch [3/10], Loss: 0.1224
Epoch [4/10], Loss: 0.0409
Epoch [5/10], Loss: 0.0115
Epoch [6/10], Loss: 0.0042
Epoch [7/10], Loss: 0.0026
Epoch [8/10], Loss: 0.0012
Epoch [9/10], Loss: 0.0003
Epoch [10/10], Loss: 0.0001
Accuracy for fold 1: 80.00%
FOLD 2
--------------------------------
Epoch [1/10], Loss: 5.0014
Epoch [2/10], Loss: 0.4919
Epoch [3/10], Loss: 0.2520
Epoch [4/10], Loss: 0.0658
Epoch [5/10], Loss: 0.0097
Epoch [6/10], Loss: 0.0016
Epoch [7/10], Loss: 0.0528
Epoch [8/10], Loss: 0.0018
Epoch [9/10], Loss: 0.0750
Epoch [10/10], Loss: 0.0220
Accuracy fo

CNN(K-fold 미적용)

In [10]:
# 라이브러리 임포트
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

import torch.nn.functional as F

# 데이터 전처리
mean = 0.5
sigma = 0.5
trans = transforms.Compose([
    transforms.Resize((300, 300)),
    transforms.ToTensor(),
    transforms.Grayscale(),
    transforms.Normalize(mean=[mean], std=[sigma])
])
trainset = datasets.ImageFolder(root='./OX_Dataset/Dataset', transform=trans)

# 데이터 분할
train_data, test_data = train_test_split(trainset, test_size=0.2, shuffle=True)

trainloader = DataLoader(train_data, batch_size=10, shuffle=True)
testloader = DataLoader(test_data, batch_size=10, shuffle=True)

# 모델 정의
class Classification(nn.Module):
    def __init__(self):
        super(Classification, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 75 * 75, 512)  # Calculate the correct flattened size
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 2)  # 2개의 클래스 (OX)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 75 * 75)  # Flatten to the correct size
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 모델 초기화
model = Classification()

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 훈련 루프
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in trainloader:
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(trainloader):.4f}')

# 모델 평가
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy: {accuracy:.2f}%')

Epoch [1/10], Loss: 2.0516
Epoch [2/10], Loss: 0.2316
Epoch [3/10], Loss: 0.0382
Epoch [4/10], Loss: 0.0022
Epoch [5/10], Loss: 0.0004
Epoch [6/10], Loss: 0.0001
Epoch [7/10], Loss: 0.0001
Epoch [8/10], Loss: 0.0000
Epoch [9/10], Loss: 0.0000
Epoch [10/10], Loss: 0.0000
Accuracy: 87.50%
